In [1]:
import librosa
import numpy as np
from numpy import newaxis
import os

In [2]:
file_dir = os.getcwd()

def getDatafile(file_dir):

    test_list = []
    test_label = [] 
    valide_list = []
    valide_label = []
    train_list = []
    train_label = []
    #定义存放各类别数据和对应标签的列表，列表名对应你所需要分类的列别名

    file_list = []   #liste des files de wav
    dic_list = [] # dictionnaire de base des données
    path_list=os.listdir(file_dir)

    for wav_dir in path_list:
        name = wav_dir.split(sep='.')
        if(len(name) == 1) :
            for wav in os.listdir(file_dir+'/'+wav_dir): 
                name_wav = wav.split(sep='.')
                if(len(name_wav) == 2 and name_wav[1] == 'wav' ):
                    file_list.append(wav_dir+'/'+wav)
                    if not(wav_dir in dic_list):
                        dic_list.append(wav_dir)

    print(len(file_list))
    print(dic_list)

    index = {}
    for i in range(len(dic_list)):
        index[dic_list[i]] = i
    #Creates the reverse table that maps labels names to their index

    print(index)
    
    
    #Creates the test_list
    for line in open('testing_list.txt','r'):
        test_list.append(line.strip('\n'))

    perm = np.random.permutation(len(test_list))
    test = list()
    for i in range(len(perm)):
        test.append(test_list[perm[i]])
        label = test_list[perm[i]].strip('\n').split(sep='/')[0]
        test_label.append(index[label])


    #Creates the valide_list
    for line in open('validation_list.txt','r'):
        valide_list.append(line.strip('\n'))
    perm = np.random.permutation(len(valide_list))
    valide = list()
    for i in range(len(perm)):
        valide.append(valide_list[perm[i]])
        label = valide_list[perm[i]].strip('\n').split(sep='/')[0]
        valide_label.append(index[label])


    #Creates the train_list
    for line in file_list:
        if not (line in test_list ):
            if not (line in valide_list):
                train_list.append(line)
    perm = np.random.permutation(len(train_list))
    train = list()
    for i in range(len(perm)):
        train.append(train_list[perm[i]])
        label = train_list[perm[i]].split(sep='/')[0]
        train_label.append(index[label])
        
        
    return index, train, train_label, valide, valide_label, test, test_label

In [4]:
index, train, train_label, valide, valide_label, test, test_label = getDatafile(file_dir)
    

105829
['right', 'eight', 'cat', 'tree', 'backward', 'learn', 'bed', 'happy', 'go', 'dog', 'no', 'wow', 'follow', 'nine', 'left', 'stop', 'three', 'sheila', 'one', 'bird', 'zero', 'seven', 'up', 'visual', 'marvin', 'two', 'house', 'down', 'six', 'yes', 'on', 'five', 'forward', 'off', 'four']
{'right': 0, 'eight': 1, 'cat': 2, 'tree': 3, 'backward': 4, 'learn': 5, 'bed': 6, 'happy': 7, 'go': 8, 'dog': 9, 'no': 10, 'wow': 11, 'follow': 12, 'nine': 13, 'left': 14, 'stop': 15, 'three': 16, 'sheila': 17, 'one': 18, 'bird': 19, 'zero': 20, 'seven': 21, 'up': 22, 'visual': 23, 'marvin': 24, 'two': 25, 'house': 26, 'down': 27, 'six': 28, 'yes': 29, 'on': 30, 'five': 31, 'forward': 32, 'off': 33, 'four': 34}


In [39]:
dataset = []

nn_train = len(train)
nx = 128
ny = 32
dataset  = np.zeros((nn_train,1,nx,ny))

i = 0 
for file in train :
    y, sr = librosa.load(file, sr=None)
    melspec = librosa.feature.melspectrogram(y, sr, n_fft=1024, hop_length=512, n_mels=128)
    logmelspec = librosa.power_to_db(melspec)
    if logmelspec.size < 4096:
        t=4096-logmelspec.size
        a=np.mean(logmelspec)
        z=np.zeros(t)
        z.fill(a)
#         print(tmp.shape)
        logmelspec=(np.append(logmelspec,z))
    elif logmelspec.size > 4096:
        logmelspec=logmelspec[:4096]
        
    #print(len(logmelspec))
        
    if(len(logmelspec) != 128):    
        logmelspec = np.reshape(logmelspec, (128, 32)) 
    
    logmelspec = (logmelspec - np.min(logmelspec) )/(np.max(logmelspec)  - np.min(logmelspec))
    
    logmelspec = logmelspec[newaxis, :, :]
    #dataset.append(logmelspec)
    dataset[i] = logmelspec
    if( i%1000 == 0):
            print(i,file)
            print(train_label[i])
    i+=1

0 forward/6347b393_nohash_0.wav
32
1000 zero/5e1b34a6_nohash_1.wav
20
2000 one/54b6d355_nohash_0.wav
18
3000 three/f0ebef1b_nohash_1.wav
16
4000 learn/1b18600d_nohash_0.wav
5
5000 one/8a28231e_nohash_0.wav
18
6000 one/98f4f088_nohash_0.wav
18
7000 on/c50f55b8_nohash_17.wav
30
8000 go/64df20d8_nohash_1.wav
8
9000 right/61e2f74f_nohash_2.wav
0
10000 yes/f33660af_nohash_0.wav
29
11000 one/00f0204f_nohash_0.wav
18
12000 sheila/e04d7130_nohash_0.wav
17
13000 one/d394ef8e_nohash_2.wav
18
14000 three/5fc3ed24_nohash_1.wav
16
15000 five/a583c5b0_nohash_0.wav
31
16000 happy/f6617a86_nohash_0.wav
7
17000 wow/4c13fe25_nohash_0.wav
11
18000 sheila/d85270c1_nohash_0.wav
17
19000 off/85851131_nohash_2.wav
33
20000 dog/de89e2ca_nohash_0.wav
9
21000 house/b76f6088_nohash_0.wav
26
22000 eight/5f8e50a0_nohash_0.wav
1
23000 bed/ae04cdbe_nohash_0.wav
6
24000 up/47565088_nohash_0.wav
22
25000 one/f0522ff4_nohash_2.wav
18
26000 off/ef3367d9_nohash_2.wav
33
27000 seven/f8ba7c0e_nohash_2.wav
21
28000 on/30a09

In [40]:
testset = []

nn_test = len(test)
nx = 128
ny = 32
testset  = np.zeros((nn_test,1,nx,ny))

i = 0 
for file in test :
    y, sr = librosa.load(file, sr=None)
    melspec = librosa.feature.melspectrogram(y, sr, n_fft=1024, hop_length=512, n_mels=128)
    logmelspec = librosa.power_to_db(melspec)
    if logmelspec.size < 4096:
        t=4096-logmelspec.size
        a=np.mean(logmelspec)
        z=np.zeros(t)
        z.fill(a)
#         print(tmp.shape)
        logmelspec=(np.append(logmelspec,z))
    elif logmelspec.size > 4096:
        logmelspec=logmelspec[:4096]
        
    if(len(logmelspec) != 128):    
          logmelspec = np.reshape(logmelspec, (128, 32)) 
    
    logmelspec = (logmelspec - np.min(logmelspec) )/(np.max(logmelspec)  - np.min(logmelspec))
    #print(logmelspec)
    logmelspec = logmelspec[newaxis, :, :]
    #dataset.append(logmelspec)
    testset[i] = logmelspec
    if( i%1000 == 0):
            print(i,file)
            print(test_label[i])
    i+=1
    
    #if(i>11000): print(file)

0 one/bb05582b_nohash_4.wav
18
1000 right/50033893_nohash_1.wav
0
2000 left/b49caed3_nohash_2.wav
14
3000 six/5ff3f9a1_nohash_1.wav
28
4000 stop/893705bb_nohash_6.wav
15
5000 two/bb05582b_nohash_0.wav
25
6000 zero/e1469561_nohash_2.wav
20
7000 nine/9a7c1f83_nohash_4.wav
13
8000 two/af7a8296_nohash_2.wav
25
9000 left/692a88e6_nohash_2.wav
14
10000 backward/5170b77f_nohash_4.wav
4
11000 three/6736bc64_nohash_0.wav
16


In [41]:
import torch
import torch.utils.data
from torch.autograd import Variable # torch 中 Variable 模块
import torchvision
import torchvision.transforms as transforms


#train_tensor = transform1(dataset)
train_tensor = torch.tensor(dataset).float()

target_tensor = torch.tensor(train_label)

print(type(train_tensor))

#print(target_tensor[0])
torch_dataset = torch.utils.data.TensorDataset(train_tensor, target_tensor)

trainloader = torch.utils.data.DataLoader(torch_dataset, batch_size=2,
                                          shuffle=True, num_workers=2)


<class 'torch.Tensor'>


In [43]:
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images)

images1, labels1 = dataiter.next()

#print(np.max(images1[0]))


tensor([[[[0.5007, 0.5536, 0.5257,  ..., 0.4672, 0.5024, 0.5025],
          [0.5354, 0.5740, 0.5765,  ..., 0.4846, 0.5019, 0.3655],
          [0.3113, 0.4641, 0.5609,  ..., 0.5033, 0.5235, 0.4922],
          ...,
          [0.3485, 0.3485, 0.3485,  ..., 0.3485, 0.3485, 0.3485],
          [0.3485, 0.3485, 0.3485,  ..., 0.3485, 0.3485, 0.3485],
          [0.3485, 0.3485, 0.3485,  ..., 0.3485, 0.3485, 0.3485]]],


        [[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.4903,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.4807,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.4016,  ..., 0.0000, 0.0000, 0.0000]]]])


Process Process-26:
Process Process-25:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/envs/python3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/anaconda3/envs/python3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/anaconda3/envs/py

In [44]:
test_tensor = torch.tensor(testset).float()
target_test = torch.tensor(test_label)

#print(test_tensor[0][0][0])

#print(target_tensor[0])
torch_testset = torch.utils.data.TensorDataset(test_tensor, target_test)


#print(torch_testset[0])
testloader = torch.utils.data.DataLoader(torch_testset, batch_size=2,
                                          shuffle=True, num_workers=2)

print(testloader)

In [55]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,16,7)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 13 * 1, 35)
        #self.fc2 = nn.Linear(256, 256)
        #self.fc3 = nn.Linear(256, 35)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        #print(x[0])
        x = x.view(-1, 64 * 13 * 1)
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc1(x)
        return x


net = Net()

In [56]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)

In [106]:
from torch.autograd import Variable # torch 中 Variable 模块

for epoch in range(30):  # loop over the dataset multiple times 

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs = Variable(data[0])
        labels = Variable(data[1])
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs)
        # _,predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.826
[1,  4000] loss: 0.853
[1,  6000] loss: 0.823
[1,  8000] loss: 0.869
[1, 10000] loss: 0.833
[1, 12000] loss: 0.834
[1, 14000] loss: 0.860
[1, 16000] loss: 0.885
[1, 18000] loss: 0.884
[1, 20000] loss: 0.844
[1, 22000] loss: 0.857
[1, 24000] loss: 0.835
[1, 26000] loss: 0.844
[1, 28000] loss: 0.841
[1, 30000] loss: 0.840
[1, 32000] loss: 0.859
[1, 34000] loss: 0.859
[1, 36000] loss: 0.847
[1, 38000] loss: 0.827
[1, 40000] loss: 0.857
[1, 42000] loss: 0.814
[2,  2000] loss: 0.862
[2,  4000] loss: 0.821
[2,  6000] loss: 0.837
[2,  8000] loss: 0.818
[2, 10000] loss: 0.842
[2, 12000] loss: 0.855
[2, 14000] loss: 0.838
[2, 16000] loss: 0.842
[2, 18000] loss: 0.816
[2, 20000] loss: 0.853
[2, 22000] loss: 0.851
[2, 24000] loss: 0.813
[2, 26000] loss: 0.827
[2, 28000] loss: 0.849
[2, 30000] loss: 0.804
[2, 32000] loss: 0.853
[2, 34000] loss: 0.834
[2, 36000] loss: 0.811
[2, 38000] loss: 0.820
[2, 40000] loss: 0.813
[2, 42000] loss: 0.826
[3,  2000] loss: 0.818
[3,  4000] 

In [122]:
dataiter = iter(testloader)

inputs, labels = dataiter.next()

print(labels)

outputs = net(inputs)

_,predicted = torch.max(outputs, 1)

print(predicted)


tensor([30,  0])
tensor([30,  0])


In [114]:
corret = 0
summ = 0
for i, data in enumerate(trainloader, 0):
    inputs = Variable(data[0])
    labels = Variable(data[1])
    outputs = net(inputs)

    _,predicted = torch.max(outputs, 1)
    
    if (predicted[0] == labels[0] ) : corret += 1
    if(len(predicted) == 2):
        if (predicted[1] == labels[1] ) : corret += 1
        
    summ += 2
    
    if (i%100 == 0 ) : print(f"--> correct {corret} : \t{round(corret/summ*100)}%")

--> correct 2 : 	100%
--> correct 162 : 	80%
--> correct 309 : 	77%
--> correct 478 : 	79%
--> correct 633 : 	79%
--> correct 788 : 	79%
--> correct 947 : 	79%
--> correct 1101 : 	79%
--> correct 1250 : 	78%
--> correct 1419 : 	79%
--> correct 1585 : 	79%
--> correct 1752 : 	80%
--> correct 1916 : 	80%
--> correct 2070 : 	80%
--> correct 2228 : 	80%
--> correct 2382 : 	79%
--> correct 2541 : 	79%
--> correct 2702 : 	79%
--> correct 2859 : 	79%
--> correct 3017 : 	79%
--> correct 3186 : 	80%
--> correct 3341 : 	80%
--> correct 3498 : 	79%
--> correct 3659 : 	80%
--> correct 3811 : 	79%
--> correct 3967 : 	79%
--> correct 4113 : 	79%
--> correct 4264 : 	79%
--> correct 4422 : 	79%
--> correct 4575 : 	79%
--> correct 4731 : 	79%
--> correct 4885 : 	79%
--> correct 5039 : 	79%
--> correct 5197 : 	79%
--> correct 5352 : 	79%
--> correct 5519 : 	79%
--> correct 5673 : 	79%
--> correct 5828 : 	79%
--> correct 5977 : 	79%
--> correct 6133 : 	79%
--> correct 6288 : 	79%
--> correct 6444 : 	79%


Process Process-189:


--> correct 45133 : 	78%


KeyboardInterrupt
Process Process-190:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/envs/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/envs/python3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/anaconda3/envs/python3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/anaconda3/envs/pyt

Traceback (most recent call last):
  File "/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-114-12b7fe07269e>", line 3, in <module>
    for i, data in enumerate(trainloader, 0):
  File "/anaconda3/envs/python3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 330, in __next__
    idx, batch = self._get_batch()
  File "/anaconda3/envs/python3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 309, in _get_batch
    return self.data_queue.get()
  File "/anaconda3/envs/python3/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/anaconda3/envs/python3/lib/python3.6/multiprocessing/synchronize.py", line 99, in __exit__
    return self._semlock.__exit__(*args)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call 

KeyboardInterrupt: 

In [113]:
corret = 0
summ = 0
for i, data in enumerate(testloader, 0):
    inputs = Variable(data[0])
    labels = Variable(data[1])
    outputs = net(inputs)

    _,predicted = torch.max(outputs, 1)
    
    if (predicted[0] == labels[0] ) : corret += 1
    if(len(predicted) == 2):
        if (predicted[1] == labels[1] ) : corret += 1  
        
    summ += 2
    
    if (i%100 == 0 ) : print(f"--> correct {corret} : \t{round(corret/summ*100)}%")
    
    

--> correct 2 : 	100%
--> correct 153 : 	76%
--> correct 302 : 	75%
--> correct 456 : 	76%
--> correct 606 : 	76%
--> correct 753 : 	75%
--> correct 904 : 	75%
--> correct 1058 : 	75%
--> correct 1212 : 	76%
--> correct 1361 : 	76%
--> correct 1510 : 	75%
--> correct 1667 : 	76%
--> correct 1804 : 	75%
--> correct 1947 : 	75%
--> correct 2088 : 	75%
--> correct 2234 : 	74%
--> correct 2383 : 	74%
--> correct 2528 : 	74%
--> correct 2678 : 	74%
--> correct 2826 : 	74%
--> correct 2975 : 	74%
--> correct 3132 : 	75%
--> correct 3283 : 	75%
--> correct 3433 : 	75%
--> correct 3586 : 	75%
--> correct 3739 : 	75%
--> correct 3901 : 	75%
--> correct 4055 : 	75%
--> correct 4214 : 	75%
--> correct 4367 : 	75%
--> correct 4521 : 	75%
--> correct 4662 : 	75%
--> correct 4830 : 	75%
--> correct 4977 : 	75%
--> correct 5129 : 	75%
--> correct 5277 : 	75%
--> correct 5430 : 	75%
--> correct 5582 : 	75%
--> correct 5725 : 	75%
--> correct 5871 : 	75%
--> correct 6010 : 	75%
--> correct 6168 : 	75%


In [117]:
model_file = 'model_30.pth'
torch.save(Net, model_file)
print('Saved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file\n')
    

Saved model to model_30.pth. You can run `python evaluate.py --model model_30.pth` to generate the Kaggle formatted csv file



/anaconda3/envs/python3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
